# 회고

1. 3가지 데이터셋을 대상으로 학습과 테스트를 수행했다.
   사용한 데이터셋: 'load_digits', 'load_wine', 'load_breast_cancer'  
   
   
2. 5가지 모델을 평가하기 위해 교차검증(cross-validation)을 사용하였다.
   학습 모델: 'Decision Tree','Random Forest','SVM','SGD Classifier','Logistic Regression'  
   
   
3. 교차검증을 사용한 이유는 몇 가지 이유가 있다.
   a. 편의: 개별 테스트로 분리하지 않고 반복문으로 돌려서 한 번에 전체 모델의 테스트 결과 얻기
   b. 비교의 공정성: 모든 모델을 같은 조건에서 평가하기
   c. 안정적인 결과: 폴드로 나눠서 여러번 테스트한 점수의 평균을 받기 때문에 일관성있는 점수 획득  
   

4. 평균을 산출한 이유는 기준이 되는 하나의 결과값으로 모델의 성능을 비교하기 위함이다.  


5. 위 테스트 결과로 각 데이터셋에 어떤 모델을 사용하는 것이 적합한지 판별했다.
   결과는 다음과 같다.   
   (아래 'Conclusion'과 동일 내용)  
   

    load_digits: SVC
    load_wine: Random Forest Classifier
    load_breast_cancer: Random Forest Classifier  
 
6. 각 데이터셋에 대해 데이터를 이해하는 과정은 
   

In [2]:
import sklearn
import numpy as np
import pandas as pd
print(sklearn.__version__)

1.0


In [3]:
# import metrics and split tool
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# classifier algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

# import datasets
from sklearn.datasets import load_digits
from sklearn.datasets import load_wine
from sklearn.datasets import load_breast_cancer

In [4]:
# created a dictionary of models in order to run them in a loop
models = {'logistic_regression':LogisticRegression,
          'SGDClassifier': SGDClassifier,
          'SVC':SVC,
          'RandomForestClassifier':RandomForestClassifier,
          'DecisionTreeClassifier':DecisionTreeClassifier
          }

In [5]:
# dictionary of datasets
# each dataset returns X(data) and y(target) value
datasets = {'digits':load_digits(return_X_y=True),
            'wine':load_wine(return_X_y=True),
            'cancer':load_breast_cancer(return_X_y=True)}

In [6]:
# accuracy of each models saved in the dictionary
scores = {'digits':{},'wine':{},'cancer':{}} 

In [27]:
# checking what values each data returns (expected to be X and y)
# load_digits(return_X_y=True)[0] #index [0] = data, [1] = target

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [17]:
# Cross_validation loop to automate model training 
# accuracy score is saved in the <score> dictionary
# take the average of each model's test evaluation result
'''
cross_val_score(args)

estimator --ML model to evaluate
X --feature
y --target
n_jobs --number of CPU cores
cv --number of folds

returns:
an array of scores, one for each fold
'''
for experiment in scores:
    for model in models:
        scores[experiment][model] = np.mean(cross_val_score(estimator=models[model](),
                                                   X = datasets[experiment][0],
                                                   y = datasets[experiment][1],
                                                   n_jobs = -1, #all available
                                                   cv = 5))#default

# call the dictionary to see if the results are saved
scores

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

{'digits': {'logistic_regression': 0.9137650882079852,
  'SGDClassifier': 0.9065428659857628,
  'SVC': 0.9632838130609718,
  'RandomForestClassifier': 0.9393577839678118,
  'DecisionTreeClassifier': 0.7818987929433612},
 'wine': {'logistic_regression': 0.9555555555555555,
  'SGDClassifier': 0.6073015873015872,
  'SVC': 0.6634920634920635,
  'RandomForestClassifier': 0.9777777777777779,
  'DecisionTreeClassifier': 0.8598412698412699},
 'cancer': {'logistic_regression': 0.935010091600683,
  'SGDClassifier': 0.8787144853283652,
  'SVC': 0.9121720229777983,
  'RandomForestClassifier': 0.9630957925787922,
  'DecisionTreeClassifier': 0.9173575531749728}}

In [18]:
# find the best performing classifier 
df_results_mean = pd.DataFrame(scores)
df_results_mean

,digits,wine,cancer
logistic_regression,0.913765,0.955556,0.935010
SGDClassifier,0.906543,0.607302,0.878714
SVC,0.963284,0.663492,0.912172
RandomForestClassifier,0.939358,0.977778,0.963096
DecisionTreeClassifier,0.781899,0.859841,0.917358


In [19]:
# sorts the result values of 'x' column, from the lowest(top) to the highest.
df_results_mean.sort_values(by='cancer')

,digits,wine,cancer
SGDClassifier,0.906543,0.607302,0.878714
SVC,0.963284,0.663492,0.912172
DecisionTreeClassifier,0.781899,0.859841,0.917358
logistic_regression,0.913765,0.955556,0.935010
RandomForestClassifier,0.939358,0.977778,0.963096


# Conclusion
Best evaluation model for each datasets are:
load_digits: SVC
load_wine: Random Forest Classifier
load_breast_cancer: Random Forest Classifier

In [14]:
# Cross_validation test_2 in order to compare what results we get without mean

for experiment in scores:
    for model in models:
        scores[experiment][model] = cross_val_score(estimator=models[model](),
                                                   X = datasets[experiment][0],
                                                   y = datasets[experiment][1],
                                                   n_jobs = -1, #all available
                                                   cv = 5) #default

# call the dictionary to see if the results are saved
scores

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

{'digits': {'logistic_regression': array([0.92222222, 0.86944444, 0.94150418, 0.93871866, 0.89693593]),
  'SGDClassifier': array([0.89722222, 0.88333333, 0.94986072, 0.95821727, 0.89415042]),
  'SVC': array([0.96111111, 0.94444444, 0.98328691, 0.98885794, 0.93871866]),
  'RandomForestClassifier': array([0.93055556, 0.90555556, 0.9637883 , 0.9637883 , 0.92200557]),
  'DecisionTreeClassifier': array([0.77777778, 0.725     , 0.81337047, 0.83286908, 0.78272981])},
 'wine': {'logistic_regression': array([0.88888889, 0.94444444, 0.94444444, 1.        , 1.        ]),
  'SGDClassifier': array([0.66666667, 0.41666667, 0.66666667, 0.62857143, 0.77142857]),
  'SVC': array([0.63888889, 0.61111111, 0.63888889, 0.68571429, 0.74285714]),
  'RandomForestClassifier': array([1.        , 0.94444444, 1.        , 0.97142857, 1.        ]),
  'DecisionTreeClassifier': array([0.91666667, 0.88888889, 0.83333333, 0.91428571, 0.85714286])},
 'cancer': {'logistic_regression': array([0.92105263, 0.93859649, 0.9561

In [16]:
df_results = pd.DataFrame(scores)
df_results

,digits,wine,cancer
logistic_regression,"[0.9222222222222223, 0.8694444444444445, 0.941...","[0.8888888888888888, 0.9444444444444444, 0.944...","[0.9210526315789473, 0.9385964912280702, 0.956..."
SGDClassifier,"[0.8972222222222223, 0.8833333333333333, 0.949...","[0.6666666666666666, 0.4166666666666667, 0.666...","[0.7894736842105263, 0.9122807017543859, 0.903..."
SVC,"[0.9611111111111111, 0.9444444444444444, 0.983...","[0.6388888888888888, 0.6111111111111112, 0.638...","[0.8508771929824561, 0.8947368421052632, 0.929..."
RandomForestClassifier,"[0.9305555555555556, 0.9055555555555556, 0.963...","[1.0, 0.9444444444444444, 1.0, 0.9714285714285...","[0.9298245614035088, 0.9473684210526315, 0.991..."
DecisionTreeClassifier,"[0.7777777777777778, 0.725, 0.8133704735376045...","[0.9166666666666666, 0.8888888888888888, 0.833...","[0.9122807017543859, 0.9122807017543859, 0.921..."
